In [46]:
import cdsapi
import threading
import time
from tqdm import tqdm

In [48]:
dataset1 = 'reanalysis-era5-single-levels'
data_year = [  # the target years
    '2022',
    '2023',
]
data_month = [  # the target months
    '01',
    '02',
    '03',
    '04',
    '05',
    '06',
    '07',
    '08',
    '09',
    '10',
    '11',
    '12',
]
data_time = [  # the target times_UTC
    '00:00',
    '01:00',
    '02:00',
    '03:00',
    '04:00',
    '05:00',
    '06:00',
    '07:00',
    '08:00',
    '09:00',
    '10:00',
    '11:00',
    '12:00',
    '13:00',
    '14:00',
    '15:00',
    '16:00',
    '17:00',
    '18:00',
    '19:00',
    '20:00',
    '21:00',
    '22:00',
    '23:00',
]
variable_list_1 = [
    '10m_u_component_of_wind',
    '10m_v_component_of_wind',
    '2m_dewpoint_temperature',
    '2m_temperature',
    'instantaneous_10m_wind_gust',
    'mean_sea_level_pressure',
    'skin_temperature',
    'surface_pressure',
    'total_precipitation',
]

download_times = {}  # Dictionary to store download times for each task
c = cdsapi.Client()

In [49]:
def is_leap_year(year):
    year = int(year)  # Convert the string to integer
    return (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0)


def days_check(year, month):
    year = int(year)  # Convert the string to integer
    days_by_month = {
        "01": 31,  # January
        "02": 29 if is_leap_year(year) else 28,  # February
        "03": 31,  # March
        "04": 30,  # April
        "05": 31,  # May
        "06": 30,  # June
        "07": 31,  # July
        "08": 31,  # August
        "09": 30,  # September
        "10": 31,  # October
        "11": 30,  # November
        "12": 31,  # December
    }

    return [str(day).zfill(2) for day in range(1, days_by_month.get(month, 0) + 1)]


def era5_get_data(c, dataset, variable_list, year, month):
    # c: api_server
    # dataset: target dataset
    # variable_list: the target variable
    try:
        start_time = time.time()  # Record start time
        filename = f'era5_single_level_{year}_{month}.nc'
        c.retrieve(
            dataset,
            {
                'product_type': 'reanalysis',
                'format': 'netcdf',
                'variable': variable_list,
                'year': year,
                'month': month,
                'day': days_check(year, month),
                'time': data_time,
                # 'format': 'netcdf.zip',
                'area': [
                    61,
                    -8,
                    50,
                    2,
                ],  # the UK range
            },
            filename,
        )
        end_time = time.time()  # Record end time
        download_time = end_time - start_time
        download_times[(year, month)] = download_time

        print(f'{filename} done!')
        print(f'Download time: {download_time:.3f} s')

    except Exception as e:
        print(f'Error downloading {filename}: {e}\n')


# Mutiple threads module for accelerating


def thread_function(year, month):
    era5_get_data(c, dataset1, variable_list_1, year, month)

In [50]:
threads = []
for i in data_year:
    for j in tqdm(data_month):
        thread = threading.Thread(target=thread_function, args=(i, j))
        threads.append(thread)
        thread.start()

for thread in threads:
    thread.join()

# Calculate and print total download times
total_download_time = sum(download_times.values())
print(f'Total download time: {total_download_time:.2f} seconds\n')

100%|████████████████████████████████████████████| 12/12 [00:00<00:00, 886.85it/s]
2023-08-14 14:02:48,394 INFO Welcome to the CDS
2023-08-14 14:02:48,397 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-08-14 14:02:48,411 INFO Welcome to the CDS
2023-08-14 14:02:48,412 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-08-14 14:02:48,419 INFO Welcome to the CDS
2023-08-14 14:02:48,419 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-08-14 14:02:48,421 INFO Welcome to the CDS
2023-08-14 14:02:48,422 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2023-08-14 14:02:48,430 INFO Welcome to the CDS
2023-08-14 14:02:48,430 INFO Welcome to the CDS
2023-08-14 14:02:48,430 INFO Welcome to the CDS
2023-08-14 14:02:48,431 INFO Welcome to the CDS
2023-08-14 14:02:

era5_single_level_2023_08.nc done!
Download time: 52.810 s


2023-08-14 14:04:04,003 INFO Request is queued
2023-08-14 14:04:04,005 INFO Request is running
2023-08-14 14:04:04,006 INFO Request is queued
2023-08-14 14:04:04,006 INFO Request is queued
2023-08-14 14:04:04,007 INFO Request is queued
2023-08-14 14:04:04,011 INFO Request is queued
2023-08-14 14:04:04,011 INFO Request is queued
2023-08-14 14:04:04,012 INFO Request is queued
2023-08-14 14:04:04,012 INFO Request is queued
2023-08-14 14:04:04,013 INFO Request is running
2023-08-14 14:04:04,092 INFO Request is queued
2023-08-14 14:04:42,524 INFO Request is running
2023-08-14 14:04:42,526 INFO Request is running
2023-08-14 14:04:42,585 INFO Request is queued
2023-08-14 14:05:40,300 INFO Request is running
2023-08-14 14:05:40,301 INFO Request is running
2023-08-14 14:05:40,352 INFO Request is queued
2023-08-14 14:07:07,710 INFO Request is running
2023-08-14 14:07:07,747 INFO Request is queued
2023-08-14 14:09:08,212 INFO Request is completed
2023-08-14 14:09:08,213 INFO Downloading https://d

era5_single_level_2022_06.nc done!
Download time: 384.359 s



 37%|██████████████▉                         | 8.77M/23.6M [00:03<00:03, 4.94MB/s]
 40%|████████████████                        | 9.48M/23.6M [00:03<00:02, 5.66MB/s]
 43%|█████████████████                       | 10.1M/23.6M [00:03<00:02, 5.85MB/s]
 46%|██████████████████▎                     | 10.8M/23.6M [00:03<00:02, 6.17MB/s]
 48%|███████████████████▎                    | 11.4M/23.6M [00:03<00:02, 6.26MB/s]
 51%|████████████████████▍                   | 12.0M/23.6M [00:03<00:01, 6.38MB/s]
 54%|█████████████████████▍                  | 12.6M/23.6M [00:03<00:01, 6.06MB/s]
 56%|██████████████████████▌                 | 13.3M/23.6M [00:03<00:01, 6.34MB/s]
 59%|███████████████████████▋                | 13.9M/23.6M [00:03<00:01, 6.23MB/s]
 62%|████████████████████████▊               | 14.6M/23.6M [00:04<00:01, 6.51MB/s]
 65%|█████████████████████████▊              | 15.2M/23.6M [00:04<00:01, 6.47MB/s]
 68%|███████████████████████████             | 15.9M/23.6M [00:04<00:01, 6.68MB/s]
 71

...

era5_single_level_2022_03.nc done!
Download time: 2428.382 s


2023-08-14 14:45:13,163 INFO Request is running
2023-08-14 14:45:13,196 INFO Request is completed
2023-08-14 14:45:13,197 INFO Downloading https://download-0008-clone.copernicus-climate.eu/cache-compute-0008/cache/data3/adaptor.mars.internal-1692020645.6156116-521-17-a8c0e217-a19b-4e18-9680-79e50418e94e.nc to era5_single_level_2023_07.nc (23.6M)
2023-08-14 14:45:16,841 INFO Download rate 6.5M/s                                 


era5_single_level_2023_07.nc done!
Download time: 2549.597 s


2023-08-14 14:47:13,388 INFO Downloading https://download-0021.copernicus-climate.eu/cache-compute-0021/cache/data2/adaptor.mars.internal-1692020712.7534235-2974-1-40221d6e-432a-474b-9b85-364919eef477.nc to era5_single_level_2022_04.nc (22.8M)
2023-08-14 14:47:16,031 INFO Download rate 8.6M/s                                 


era5_single_level_2022_04.nc done!
Download time: 2668.826 s
Total download time: 28068.61 seconds

